In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm
import matplotlib.image as mpimg
from random import choice
from math import log
from skimage.filters import unsharp_mask
from skimage import io, morphology
from skimage import exposure
import cv2

In [7]:
def add_gaussian_noise(image, mean=0, sigma=25):
#Aggiunge rumore gaussiano all'immagine
    row, col= image.shape
    noise = np.random.normal(mean, sigma, (row, col))
    noisy_image = np.clip(image + noise, 0, 1)
    return noisy_image

def equalize_within_mask(image, mask, range_min=0, range_max=1):
    # Applica la maschera all'immagine
    masked_image = np.copy(image)
    masked_image[~mask] = 0

    # Calcola l'istogramma dell'immagine all'interno della maschera
    hist, bin_centers = exposure.histogram(masked_image)

    # Equalizza l'istogramma all'interno del range specificato
    equalized_image = exposure.rescale_intensity(masked_image, in_range='image', out_range = (range_min,range_max))

    image_to_add = np.copy(image)
    image_to_add[mask] = 0
    equalized_image = equalized_image + image_to_add

    return equalized_image

def low_pass(img,size):

    kernel = np.ones((size,size))
    kernel = kernel/(size**2)
    filtered_img = convolve2d(img, kernel, mode='same', boundary='symm')
    return filtered_img

def enchance_contrast(img,f):

    media = np.mean(img)
    diff = img - np.array(media)
    img_con = img + f*diff
    img_con[img_con < 0] = 0
    img_con[img_con > 1] = 1

    return img_con





    return new_image
def gaussian_kernel(size, sigma):
    """
    Crea un kernel gaussiano 2D.
    """
    kernel = np.fromfunction(
        lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x - size//2)**2 + (y - size//2)**2)/(2*sigma**2)),
        (size, size)
    )
    return kernel / np.sum(kernel)

def gaussian_blur(image, kernel_size, sigma):
    """
    Applica un filtro gaussiano all'immagine utilizzando il kernel definito.
    """
    # Crea il kernel gaussiano
    kernel = gaussian_kernel(kernel_size, sigma)
    
    # Applica la convoluzione tra l'immagine e il kernel gaussiano
    blurred_image = cv2.filter2D(image, -1, kernel)
    
    return blurred_image

def balance_luminosity_add(img,mask,kernel_size,difference):

    size = np.shape(img)[0]
    kernel = np.ones((kernel_size,kernel_size))

    media_im = np.mean(img[mask])

    binary_mask = np.zeros(np.shape(mask))
    binary_mask[mask] = 1

    trsh = (kernel_size**2)/2
    available_pos = []

    for i in range(0,size-kernel_size,int(kernel_size/2)):
        for j in range(0,size-kernel_size,int(kernel_size/2)):
            check_pos = np.sum(binary_mask[i:i+kernel_size,j:j+kernel_size] * kernel)
            if check_pos > trsh: 
                available_pos.append((i,j))

    new_image = np.copy(img)            

    for coords in available_pos:
        y = coords[0]
        x = coords[1]
        media_loc = np.mean(img[y:y+kernel_size,x:x+kernel_size])
        if abs(media_loc-media_im) >difference:
            correzione_l = (media_im - media_loc)/10
            patch = new_image[y:y+kernel_size,x:x+kernel_size] + correzione_l
            patch[patch < 0] = 0.
            patch[patch > 1] = 1.0
            new_image[y:y+kernel_size,x:x+kernel_size] = patch

    return new_image


def pre_process(img):
    mask_cerchio = img < 0.00784 
    img[mask_cerchio] = 1.0
    
    img_gamma = img ** log(0.5,np.mean(img[~mask_cerchio]))
    img_f = gaussian_blur(img_gamma, kernel_size=9, sigma=1)
    img_f[mask_cerchio] = np.mean(img_f[~mask_cerchio])
    img_con = enchance_contrast(img_f,-0.1)
    img_sh =  unsharp_mask(img_con, radius=18, amount=4)
    img_sh[mask_cerchio] = np.mean(img_sh[~mask_cerchio])
    img_con_2 = enchance_contrast(img_sh, 0.2)
    mask_bianchi = img_con_2 > np.percentile(img_con_2[~mask_cerchio],96)
    selem_2 = morphology.disk(24)
    closing = morphology.dilation(mask_bianchi, selem_2)
    img_con_2[closing] = img_con_2[closing] ** log(0.6,np.mean(img_con_2[closing]))
    img_e = equalize_within_mask(img_con_2,~mask_cerchio)
    img_std = (img_e - np.mean(img_e[~mask_cerchio]))/np.std(img_e[~mask_cerchio])
    img_std = (img_std - np.min(img_std[~mask_cerchio])) / (np.max(img_std[~mask_cerchio])- np.min(img_std[~mask_cerchio]))
    img_std[mask_cerchio] = 1.0
    return img_std

PREPROCESS

In [3]:
path_input = 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\univesita\\Elaborazione immagini biomediche\\RETINA\\Original'
path_output = 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Prova'

In [15]:
lista_im = os.listdir(path_input)
medie_rumore = [-0.1,0,0,0, 0.1]
sigme_rumore = [0.07,0.05, 0.02]
aspect_ratio = [1,1,3/4,9/16, 2/3]
sizes = [3500, 2048, 1024,512]


for im in lista_im:
    image_path = os.path.join(path_input, im)
    img_o = Image.open(image_path)
    dim = choice(sizes)
    img_rsz = img_o.resize((dim, int(dim * choice(aspect_ratio))))
    img_g = np.array(img_rsz)[:,:,1]/255
    #img_p= pre_process(img_g)           # Penso che il preprocess vada fatto a questo punto
    img_no = add_gaussian_noise(img_g,choice(medie_rumore), choice(sigme_rumore))  #CAMBIARE img_g con img_p se preprocess
    img_8 = (img_no*255).astype(np.uint8)
    img_to_save = Image.fromarray(img_8)
    img_to_save.save(os.path.join(path_output,im))


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

CLUSTERING

In [18]:
#!pip install skan

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from skimage import img_as_ubyte, measure, morphology, feature, color
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from skan import Skeleton, summarize
from sklearn.model_selection import train_test_split
import shutil

SyntaxError: invalid syntax (1428816432.py, line 1)

In [ ]:
def calcola_vasi_per_tipo(img):

  if np.max(img) == 0:
    print('immagine vuota')
    return 0, 0
  skt = skeletonize(img)

  skeleton_data = Skeleton(skt)
  skel_summary = summarize(skeleton_data)

  # count how many branches have branch-type = 1 and 2
  N_J2E = len(skel_summary[skel_summary['branch-type'] == 1])
  N_J2J = len(skel_summary[skel_summary['branch-type'] == 2])

  return N_J2E, N_J2J

def calculate_glcm_features(gray_image):
    # Specify the distances and angles for GLCM
    distances = [3, 7, 11]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    # Calculate GLCM
    glcm = greycomatrix(gray_image, distances=distances, angles=angles, symmetric=True, normed=True)

    # Calculate GLCM features
    contrast = np.mean(greycoprops(glcm, 'contrast'))
    dissimilarity = np.mean(greycoprops(glcm, 'dissimilarity'))
    homogeneity = np.mean(greycoprops(glcm, 'homogeneity'))
    energy = np.mean(greycoprops(glcm, 'energy'))
    correlation = np.mean(greycoprops(glcm, 'correlation'))

    return contrast, dissimilarity, homogeneity, energy, correlation

In [ ]:
path_im = path_output
path_man = '/content/drive/MyDrive/EIM/Retina/RETINA/Ground truth'
path_constr = ''
path_test = ''

In [ ]:
list_im = os.listdir(path_im)
list_man = os.listdir(path_man)

n_e_list = []
n_j_list = []
max_size = []
asp_ratio = []

for file_im in tqdm(list_man):
  image_path_m = os.path.join(path_man,file_im)
  image_path = os.path.join(path_im,file_im)
  img = Image.open(image_path)
  img_m = Image.open(image_path_m)
  max_size.append(max(img.size[0], img.size[1]))
  asp_ratio.append(img.size[0]/img.size[1])
  pic = np.array(img_m)
  n_e, n_j = calcola_vasi_per_tipo(pic)
  n_e_list.append(n_e)
  n_j_list.append(n_j)


asp_ratio = asp_ratio -np.min(asp_ratio)/(np.max(asp_ratio)-np.min(asp_ratio))
max_size = max_size -np.min(max_size)/(np.max(max_size)-np.min(max_size))
n_e_list = n_e_list -np.min(n_e_list)/(np.max(n_e_list)-np.min(n_e_list))
n_j_list = n_j_list -np.min(n_j_list)/(np.max(n_j_list)-np.min(n_j_list))

asp_ratio = (asp_ratio - np.mean(asp_ratio))/np.std(asp_ratio)
max_size = (max_size - np.mean(max_size))/np.std(max_size)
n_e_list = (n_e_list - np.mean(n_e_list))/np.std(n_e_list)
n_j_list = (n_j_list - np.mean(n_j_list))/np.std(n_j_list)


In [ ]:
pca = PCA(n_components=2)

# Convert lists to NumPy arrays and stack them vertically
X = np.hstack((asp_ratio,max_size, n_e_list, n_j_list))  # Transpose to make it a 2D array

X_pca = pca.fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=42)
y_kmeans = kmeans.fit_predict(X_pca)

# Visualize the original data and the obtained clusters
plt.figure(figsize=(12, 5))

# Plot original data
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], cmap='viridis', edgecolor='k', s=50)
plt.title('Original Data')

# Plot data transformed with PCA
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_kmeans, cmap='viridis', edgecolor='k', s=50)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='X', s=200, label='Centroids')
plt.title('PCA + Clustering')

plt.show()

In [ ]:
inertias = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i, random_state = 0 )
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

#!!!!!!!!!!!!!!!
#MODIFICA QUA IL NUMERO DI CLUSTER
n_clusters = 5                          #<- <- <- <-

kmeans = KMeans(n_clusters=3, random_state=42)
y_kmeans = kmeans.fit_predict(X_pca)

# Visualizza i dati originali e i cluster ottenuti
plt.figure(figsize=(12, 5))

# Plot dati originali
plt.subplot(1, 2, 1)
plt.scatter(X[:,0], X[:,1], cmap='viridis', edgecolor='k', s=50)
plt.title('Dati Originali')

# Plot dati trasformati con PCA
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_kmeans, cmap='viridis', edgecolor='k', s=50)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='X', s=200, label='Centroidi')
plt.title('PCA + Clustering')
plt.legend

plt.show()

In [ ]:
pazconcluster = {}

for i in range(len(y_kmeans)):
    diz = {}
    diz['pz'] = lista_im[i]
    diz['cluster'] = y_kmeans[i]
    pazconcluster.append(diz)

df_paz = pd.DataFrame(pazconcluster)

lista_constr = []
lista_test = []
for i in range(n_clusters-1):
    df_cluster = df_paz[df_paz['cluster'] == i]
    constr, test = train_test_split(df_cluster, test_size=0.2, random_state=42)
    constr_temp = list(constr['pz'])
    test_temp = list(test['pz'])
    lista_constr = lista_constr + constr_temp
    lista_test = lista_test + test_temp

In [ ]:
for pz in lista_constr:
    percorso_file_originale = os.path.join(path_im, pz)
    percorso_destinazione = os.path.join(path_constr,pz)
    shutil.move(percorso_file_originale, percorso_destinazione)

for pz in lista_test:
    percorso_file_originale = os.path.join(path_im, pz)
    percorso_destinazione = os.path.join(path_test,pz)
    shutil.move(percorso_file_originale, percorso_destinazione)